# 贝叶斯算法

## 导入库

In [4]:
import numpy as np
import pandas as pd

## 导入数据

In [5]:
train_data = [  ['Sunny',   'Hot', 'High',  'Weak',  'No'],
                ['Sunny',   'Hot', 'High',  'Strong','No'],
                ['Overcast','Hot', 'High',  'Weak',  'Yes'],
                ['Rain',    'Mild','High',  'Weak',  'Yes'],
                ['Rain',    'Cool','Normal','Weak',  'Yes'],
                ['Rain',    'Cool','Normal','Strong','No'],
                ['Overcast','Cool','Normal','Strong','Yes'],
                ['Sunny',   'Mild','High',  'Weak',  'No'],
                ['Sunny',   'Cool','Normal','Weak',  'Yes'],
                ['Rain',    'Mild','Normal','Weak',  'Yes'],
                ['Sunny',   'Mild','Normal','Strong','Yes'],
                ['Overcast','Mild','High',  'Strong','Yes'],
                ['Overcast','Hot', 'Normal','Weak',  'Yes'],
                ['Rain',    'Mild','High',  'Strong','No']  ]

train_Data = pd.DataFrame(train_data,columns=['Outlook','Temperature','Humidity','Wind','PlayTennis'])
train_Data


,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [6]:
cols = train_Data.shape[1]
X_data = train_Data.iloc[:,:cols-1]
Y_data = train_Data.iloc[:,cols-1:]
featureNames = X_data.columns 

## 贝叶斯模型

In [13]:
def Naive_Bayes(X_data,Y_data):
    y=Y_data.values
    X=X_data.values
    y_unique=np.unique(y)
    prior_prob=np.zeros(len(y_unique))
    for i in range(len(y_unique)):
        prior_prob[i]=sum(y==y_unique[i])/len(y)
    condition_prob={} 

    for feat in featureNames:
        x_unique = list(set(X_data[feat])) 
        x_condition_prob = np.zeros((len(y_unique),len(x_unique)))
        for j in range(len(y_unique)):
            for k in range(len(x_unique)):
                x_condition_prob[j,k]=\
                sum((X_data[feat]==x_unique[k]) & (Y_data.PlayTennis==y_unique[j]))/sum(y==y_unique[j])
        x_condition_prob = pd.DataFrame(x_condition_prob,columns=x_unique,index=y_unique)
        condition_prob[feat] = x_condition_prob
    
    return prior_prob,condition_prob

## 预测函数

In [14]:
def Prediction(testData,prior,condition_prob):
    numclass = prior.shape[0] 
    numsample = testData.shape[0] 
    featureNames = testData.columns 
    post_prob = np.zeros((numsample,numclass)) 
    for k in range(numsample):
        #第k个样本p(yes),p(no)
        prob_k = np.zeros((numclass,))
        for i in range(numclass):
            pri = prior[i]
            for feat in featureNames:
                feat_val = testData[feat][k]
                cp = condition_prob[feat]
                if(i==0):
                    t='No'
                else:
                    t='Yes'
                cp_val = cp.loc[t,feat_val]
                pri *= cp_val
            prob_k[i] = pri
        prob = prob_k/np.sum(prob_k,axis = 0)
        post_prob[k,:] = prob
    return post_prob

In [16]:
prior_prob ,condition_prob = Naive_Bayes(X_data,Y_data)
test_data = [['Sunny','Cool','High','Strong']]
test_Data = pd.DataFrame(test_data,columns=['Outlook','Temperature','Humidity','Wind'])
postPrior = Prediction(test_Data,prior_prob,condition_prob)
print(postPrior)


if(postPrior[0,0]>postPrior[0,1]):
    print('We cannot play tennis')
elif (postPrior[0,0]==postPrior[0,1]):
    print('uncertainty')
else:
    print('We can play tennis')

[[0.79541735 0.20458265]]
We cannot play tennis
